# 1. Data Wrangling

<b>Load Libraries</b>

In [137]:
# data structures
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
## settings
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

<b>Load Dataset</b>

In [138]:
# dataset
df_base = pd.read_excel('../dataset/data.xlsx', sheet_name='Year 2010-2011')

<b>Peek at Data</b>

In [139]:
# shape
df_base.shape

(541910, 8)

In [140]:
# head
df_base.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [141]:
# tail
df_base.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France
541909,581587,POST,POSTAGE,1,2011-12-09 12:50:00,18.00,12680.0,France


In [142]:
# info
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


<b>Format Adjustments</b>

In [143]:
# dtype columns separation
num_cols = df_base.select_dtypes(np.number).columns.tolist()
cat_cols = df_base.select_dtypes('object').columns.tolist()

In [144]:
# cloning
df_cleaned = df_base.copy()

In [145]:
# trimming white spaces
for name in cat_cols:
    df_cleaned[name] = df_base[name].astype(str).str.strip()

In [146]:
# null strings
mask = df_cleaned[cat_cols] == ''
counts = mask.sum(axis=0)
print(f'Total null strings per column: \n{counts}')

Total null strings per column: 
Invoice        0
StockCode      0
Description    0
Country        0
dtype: int64


In [147]:
# spelling
for name in cat_cols:
    print(f'{name}: {df_cleaned[name].unique()}')

Invoice: ['536365' '536366' '536368' ... '581585' '581586' '581587']
StockCode: ['85123A' '71053' '84406B' ... '90214U' '47591b' '23843']
Description: ['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER' ... 'lost'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE']
Country: ['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


In [148]:
# low variance
low_variance = df_cleaned.nunique().to_frame('nunique')
low_variance['percentage(%)'] = (low_variance['nunique'] / len(df_cleaned)) * 100
low_variance

,nunique,percentage(%)
Invoice,25900,4.779391
StockCode,4070,0.751047
Description,4212,0.777251
Quantity,722,0.133232
InvoiceDate,23260,4.292226
Price,1630,0.300788
Customer ID,4372,0.806776
Country,38,0.007012


In [149]:
# dtypes conversion
df_cleaned[cat_cols] = df_cleaned[cat_cols].astype('category')

<b>Duplications</b>

In [150]:
# counts
mask = df_cleaned.duplicated()
counts = mask.sum()
print(f'Total duplications: {counts}')

## Conclusion: Those duplications may be errors because they have the same invoice which has to be unique. 
## Therefore,we'll choose to remove it.

Total duplications: 5268


In [151]:
# drop duplications
df_cleaned.drop_duplicates(inplace=True)

<b>Missing Values</b>

In [152]:
# counts
mask = df_cleaned.isnull()
counts = mask.sum()
print(f'Total missing values per columns: \n{counts}')

## Conclusion: Due to the need for later features creation, we'll remove those rows with null 'Customer ID'

Total missing values per columns: 
Invoice             0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135037
Country             0
dtype: int64


In [153]:
# drop
df_cleaned.dropna(axis=0, subset='Customer ID', inplace=True)

<b>Features Engineering</b>

In [154]:
# time format standardization
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'].dt.strftime('%Y-%m-%d'))

In [155]:
# drop unrelated-task features
try:
    df_cleaned.drop(['StockCode', 'Description'], axis=1, inplace=True)
except Exception as e:
    print(e)

In [156]:
# dtypes conversion 
df_cleaned['Customer ID'] = df_cleaned['Customer ID'].astype(int).astype('category')

In [157]:
# rename columns and reset index
names = ['invoice', 'quantity', 'invoice_date', 'price', 'customer_id', 'country']
df_cleaned.columns = names

try:
    df_cleaned.set_index('customer_id', inplace=True)
except Exception as e:
    print(e)

In [158]:
# time format standardization
df_cleaned['invoice_date'] = pd.to_datetime(df_cleaned['invoice_date'].dt.strftime('%Y-%m-%d'))

In [159]:
# drop invoices that are canceld
mask = ~df_cleaned['invoice'].str.startswith('C')
df_cleaned = df_cleaned.copy()[mask]

In [160]:
# 'frequency' column
freq = df_cleaned.groupby('customer_id', observed=True)['invoice'].count().to_frame('frequency').sort_index()

In [161]:
# 'monetary' column
## 'observed' parameter means that show values if it's in presence, use for 'groupby' with multiple columns as 'Grouper'
df_cleaned['total'] = df_cleaned['quantity'] * df_cleaned['price']
monetary = df_cleaned.groupby('customer_id', observed=True)['total'].sum().to_frame('monetary').sort_index()

In [202]:
monetary

,monetary
customer_id,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [204]:
# 'recency' column
mask = df_cleaned.groupby('customer_id', observed=True)['invoice_date'].rank(method='first', ascending=False) == 1
recency = (df_cleaned[mask]['invoice_date'] - min(df_cleaned[mask]['invoice_date'])).dt.days.to_frame('recency').sort_index()

In [271]:
# finalize and save df
df_rfm = pd.concat([recency, freq, monetary], ignore_index=False, axis=1)

## merging to add the 'country' column
df_to_be_merged = df_cleaned['country'].reset_index().drop_duplicates(subset='customer_id')
df_rfm = pd.merge(left=df_rfm, right=df_to_be_merged, left_index=True, right_on='customer_id').set_index('customer_id')

# saving to the separate file
df_rfm.to_csv('../dataset/rfm.csv')